## 캐싱(Caching)

LangChain은 LLM을 위한 선택적 캐싱 레이어를 제공합니다.

이는 두 가지 이유로 유용합니다.

- 동일한 완료를 여러 번 요청하는 경우 LLM 공급자에 대한 **API 호출 횟수를 줄여 비용을 절감**할 수 있습니다.
- LLM 제공업체에 대한 **API 호출 횟수를 줄여 애플리케이션의 속도를 높일 수** 있습니다.

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH04-Models")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH04-Models


모델과 프롬프트를 생성합니다


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# 모델을 생성합니다.
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template("{country} 에 대해서 200자 내외로 요약해줘")

# 체인을 생성합니다.
chain = prompt | llm

In [4]:
%%time 
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 나라로, 수도는 서울에 있다. 문화, 역사, 음식, 엔터테인먼트 등 다양한 측면에서 독특한 매력을 가지고 있다. 전통적인 한복과 김치, 불국산 등의 전통음식이 유명하며, K-pop과 드라마를 통해 글로벌한 인기를 얻고 있다. 고대부터 현대까지 다양한 역사적 사건들이 있었고, 한반도 북쪽에 북한이 있어 남북관계가 계속 주목받고 있다. 최근에는 IT기술 등의 발전으로 경제적 성장을 이루고 있으며, 한류 열풍으로 세계 각지에서 한국 문화에 대한 관심이 높아지고 있다.
CPU times: total: 93.8 ms
Wall time: 2.3 s


## InMemoryCache

인메모리 캐시를 사용하여 동일 질문에 대한 답변을 저장하고, 캐시에 저장된 답변을 반환합니다.

In [ ]:
%%time
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# 인메모리 캐시를 사용합니다. (Ram에 저장)
# 캐시를 사용하면 동일한 요청에 대해 빠른 응답을 받을 수 있습니다
# 캐시가 초기화 됩니다.
set_llm_cache(InMemoryCache())

# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한반도 동쪽에 위치한 대한민국은 수도가 서울에 있는 공화국입니다. 경제적으로 선진국으로 개발되어 있으며 세계적으로 유명한 기업들이 많이 있습니다. 주요 산업은 전자제품, 자동차, 선박, 철강 등이 있으며 문화적으로도 K-pop, K-drama 등이 대중들에게 큰 영향을 끼치고 있습니다. 또한 한국은 삼성, 현대, LG 등 글로벌 기업들이 본사를 두고 있으며 IT 기술과 각종 산업에서 선도적인 역할을 하고 있습니다. 역사적으로는 고대 시대부터 고려, 조선 왕조 등 다양한 왕조가 이어졌으며 한반도 주변 국가들과의 관계 또한 중요한 이슈입니다.
CPU times: total: 469 ms
Wall time: 3.26 s


In [6]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한반도 동쪽에 위치한 대한민국은 수도가 서울에 있는 공화국입니다. 경제적으로 선진국으로 개발되어 있으며 세계적으로 유명한 기업들이 많이 있습니다. 주요 산업은 전자제품, 자동차, 선박, 철강 등이 있으며 문화적으로도 K-pop, K-drama 등이 대중들에게 큰 영향을 끼치고 있습니다. 또한 한국은 삼성, 현대, LG 등 글로벌 기업들이 본사를 두고 있으며 IT 기술과 각종 산업에서 선도적인 역할을 하고 있습니다. 역사적으로는 고대 시대부터 고려, 조선 왕조 등 다양한 왕조가 이어졌으며 한반도 주변 국가들과의 관계 또한 중요한 이슈입니다.
CPU times: total: 0 ns
Wall time: 3.01 ms


## SQLite Cache


In [7]:
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache
import os

# 캐시 디렉토리를 생성합니다.
if not os.path.exists("cache"):
    os.makedirs("cache")

# SQLiteCache를 사용합니다.
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

In [10]:
%%time 
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 대한민국과 조선민주주의인민공화국으로 나뉜다. 대한민국은 수도인 서울을 중심으로 서비스 산업이 발달한 선진국가이며 K-pop, K-drama 등 한류 문화로 세계적인 영향력을 행사하고 있다. 조선민주주의인민공화국은 북한이라고도 불리며 이노, 북한을 대표하는 수도이다. 한반도 전체적으로 긴 역사와 전통을 자랑하며 한글을 사용한다. 정통음식으로는 김치, 불고기, 비빔밥 등이 유명하며 사람들의 민족주의가 강하다. 현재는 북한 핵 문제로 인해 국제적으로 관심을 받고 있으며 한반도평화 및 통일을 향한 노력이 계속되고 있다.
CPU times: total: 0 ns
Wall time: 4.07 ms
